In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install fanalysis

from sklearn.metrics import confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# les noms des variables

colonnes =(["IDENT", "TypeHabi", "GeoFaz_Scl", "GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Anciennete",
            "Logement_prix_m2", "Mediane_RFR", "Sexe", "Prenom_Phase_scl_H", "Prenom_Phase_scl_F",
            "Prenom_Phase_cl_H", "Prenom_Phase_cl_F", "Age_H", "Age_F", "Affinite_1", "Affinite_2",
            "Prenom_effectif", "Potentiel_prenom", "Onoma-Phase_Scl", "Onoma-Phase_CL", "Top_mon"])

In [4]:
# importation du fichier (changer le chemin d'accès)
''' Faites attention au type de fichier, au délimiteur des valeurs (ça doit être unique et non multiple),
au header et à la lecture des types d'objets (low_memory) '''

data_orig = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/new_my_data_FDFTOT21.txt", delim_whitespace="\t", header = None, names=colonnes, low_memory=False, )#, dtype='Int64')


In [5]:
''' copie superficielle (peu profonde) du dataset original pour ne pas modifier les valeurs
d'origine par erreur plus tard 
'''
data = data_orig.copy()
data.shape

(1192180, 23)

In [6]:
# visualisation des premiers lignes
data.head()

,IDENT,TypeHabi,GeoFaz_Scl,GeoFaz_cl,ImmoFaz_Scl,ImmoFaz_cl,Anciennete,Logement_prix_m2,Mediane_RFR,Sexe,Prenom_Phase_scl_H,Prenom_Phase_scl_F,Prenom_Phase_cl_H,Prenom_Phase_cl_F,Age_H,Age_F,Affinite_1,Affinite_2,Prenom_effectif,Potentiel_prenom,Onoma-Phase_Scl,Onoma-Phase_CL,Top_mon
0,10010000001,2,81,8,91,9,6,21,21,1,71,41,7,4,12,4,20,2,2,1,51,5,1.0
1,20010000006,1,75,7,43,4,6,6,13,2,71,64,7,6,12,9,4,12,4,13,12,1,1.0
2,30010000011,1,76,7,43,4,6,6,16,3,54,81,5,8,7,12,6,6,4,18,36,3,2.0
3,40010000013,3,75,7,43,4,4,6,13,1,71,36,7,3,12,11,11,19,4,6,21,2,1.0
4,50010000015,2,33,3,44,4,6,7,12,1,71,42,7,4,12,5,14,3,3,1,14,1,2.0


In [7]:
#nombre de modalités de chaque variables (dans un dataframe)
def nombre_mod(data2):
    #création d'un dataframe vide
    dff= pd.DataFrame(columns=["VARIABLE","NOMBRE_MOD", "NOMBRE_MOD_SANS_NA"])
    for i in range(data2.shape[1]): # sans compter les valeurs manquantes
        taille =len(data2.iloc[:,i].value_counts(dropna=False))
        taille_1 = len(data2.iloc[:,i].value_counts(dropna=True) )
        dff=dff.append({"VARIABLE": colonnes[i],"NOMBRE_MOD":taille, "NOMBRE_MOD_SANS_NA":taille_1 }, ignore_index=True)
    return dff

# sélection des corrélations les plus importantes
def variables_corr(tableau_corr, seuil):
    colonnes = tableau_corr.columns
    val_temp = 0 
    variables_corr_pos=[] ; variables_corr_pos_val=[]
    variables_corr_neg=[] ; variables_corr_neg_val=[]
    for i in range(tableau_corr.shape[0]):  
        for j in range(tableau_corr.shape[0]):
            if i<j: #i!=j:
                if (tableau_corr.iloc[i,j] > seuil):
                    variables_corr_pos.append(colonnes[j])
                    variables_corr_pos_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

                if (tableau_corr.iloc[i,j] <-seuil):
                    variables_corr_neg.append(colonnes[j])
                    variables_corr_neg_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

        if (len(variables_corr_pos) > 0) | (len(variables_corr_neg) >0) :
            print("variable : ", colonnes[i])
            if len(variables_corr_pos) > 0:    
                print("corr pos", variables_corr_pos)
                print("corr pos", variables_corr_pos_val)
                val_temp += 1

            if len(variables_corr_neg) >0:  
                print("corr neg", variables_corr_neg)
                print("corr neg", variables_corr_neg_val)
                val_temp += 1
            print("")
        variables_corr_pos = variables_corr_pos*0
        variables_corr_neg = variables_corr_pos*0

        variables_corr_pos_val = variables_corr_pos_val*0
        variables_corr_neg_val = variables_corr_pos_val*0
    if val_temp == 0:
        print("IL N'Y A PAS DE CORRELATION ENTRE LES VARIABLES PAR RAPPORT AU SEUIL PRIS")
        

def evaluation(model, X_train, y_train, X_test, y_test, scoring, plot):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("test set : ")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred)) 
    print("train set : ")
    print(confusion_matrix(y_train, model.predict(X_train)))  
    print(classification_report(y_train, model.predict(X_train) ))

    if plot == True:
        N, train_score, val_score = learning_curve(model, X_train, y_train, shuffle=True, cv = 4, scoring = scoring, train_sizes = np.linspace(0.1, 1, 6) )

        plt.figure(figsize = (10,6))
        plt.plot(N, train_score.mean(axis=1), label ="train_score")
        plt.plot(N, val_score.mean(axis=1), label ="cross_validation_score")
        plt.legend()
    else :
        print("GRAPHIQUE NON DEMANDE")

# Comme il n'y a pas de valeurs manquantes (à la place il y a un nombre), le seul traitement qu'on fait c'est tranformer
# le type de variable (les variables continues deviennent qualitatives)
def data_processing(df):
    """ 
    for i in range(0,len(df.columns)):    
        df[df.columns[i]].fillna(100.0, inplace=True) #On remplace les données manquantes par 100
    for i in range(0,len(df.columns)):
        print(df[df.columns[i]].value_counts(dropna=False))    
    """
    
   #Transformation de toutes les variables en facteurs         
    for i in range(0,len(df.columns)):
        df[df.columns[i]] = df[df.columns[i]].astype('category')
    return df.to_numpy() #.as_matrix()


def vingtiles(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    #print(df_tous)
    
    # pour la colonne Top_mon :
    # remplissage de la dernière colonne par 2 (VALABLE QUE POUR CE DATA, MODIFIER POUR D'AUTRES)
    data_temp_t = data_orig.copy()
    # remplacement
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(1.0,'1')
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(2.0,'2')
    data_all_complet_class = data_temp_t.fillna('2')
    #data_all_complet_class

    # jointure
    ident_class_tous = data_all_complet_class[["IDENT","Top_mon"]]
    tableau_resultats = pd.merge(df_tous,ident_class_tous)
    #tableau_resultats

    # tranches finales
    tab_cross_temp = pd.crosstab(tableau_resultats["TRANCHES"],tableau_resultats["Top_mon"])
    #print(tab_cross_temp); print("-----------------------")

    # création d'une colonne total
    tab_cross_temp["TOTAL"]=tab_cross_temp["1"]+tab_cross_temp["2"]
    
    # affichage
    if affichage ==True:        
        print(tab_cross_temp)

    # création du tableau final en éliminant la colonne 2
    #tab_pour_excel = tab_cross_temp.drop(["2"], axis=1)

    # si on ne veut pas éliminer la colonne 2
    tab_pour_excel =  tab_cross_temp
    
    # enregistrement sous Excel
    if save == True:
        tab_pour_excel.to_excel(path)

def vingtiles_ind_sans_target(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    print(df_tous)
    print(df_tous["TRANCHES"].value_counts(ascending = True))

    # GERE L'ORDRE
    patra = np.unique(df_tous.iloc[:,2], return_counts=True)
    df_tranches_sans_target = pd.DataFrame([patra[0],patra[1]]).T
    df_tranches_sans_target.columns=["TRANCHES","COUNTS"]

    # affichage
    if affichage ==True:        
        print(df_tranches_sans_target)
    
    # enregistrement sous Excel
    if save == True:
        df_tranches_sans_target.to_excel(path)
    

# élimination des variables trop corrélées (choix manuel)
def elimination_val_corr(data):
    data = data.drop(columns=[""])
    return data
          
        


In [35]:
def vingtiles_xgb(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var].values)
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    #print(df_tous)
    
    # pour la colonne Top_mon :
    # remplissage de la dernière colonne par 2 (VALABLE QUE POUR CE DATA, MODIFIER POUR D'AUTRES)
    data_temp_t = data_orig.copy()
    # remplacement
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(1.0,'1')
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(2.0,'2')
    data_all_complet_class = data_temp_t.fillna('2')
    #data_all_complet_class

    # jointure
    ident_class_tous = data_all_complet_class[["IDENT","Top_mon"]]
    tableau_resultats = pd.merge(df_tous,ident_class_tous)
    #tableau_resultats

    # tranches finales
    tab_cross_temp = pd.crosstab(tableau_resultats["TRANCHES"],tableau_resultats["Top_mon"])
    #print(tab_cross_temp); print("-----------------------")

    # création d'une colonne total
    tab_cross_temp["TOTAL"]=tab_cross_temp["1"]+tab_cross_temp["2"]
    
    # affichage
    if affichage ==True:        
        print(tab_cross_temp)

    # création du tableau final en éliminant la colonne 2
    #tab_pour_excel = tab_cross_temp.drop(["2"], axis=1)

    # si on ne veut pas éliminer la colonne 2
    tab_pour_excel =  tab_cross_temp
    
    # enregistrement sous Excel
    if save == True:
        tab_pour_excel.to_excel(path)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve, GridSearchCV

In [9]:
####################  DONNEES SANS LES VARIABLES CORR ET ENCODAGE DUMMIES        ###########################

In [10]:
# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F","Onoma-Phase_Scl",
                "Onoma-Phase_CL"],axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )
data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe IFI+
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
####################################################################################
#                            MODELE 1 : BOOSTING
####################################################################################

In [ ]:
model_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,20, 2)), 'n_estimators' : [20, 50] }

grid_AdaBoost = GridSearchCV(model_AdaBoost, param, cv = 4, scoring = "recall")
resultat_grid_AdaBoost = grid_AdaBoost.fit(train_X_s, train_Y_s)
resultat_grid_AdaBoost.best_params_
#model_AdaBoost.get_params


{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'n_estimators': 50}

In [11]:
final_model_AdaBoost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(class_weight="balanced", criterion="entropy",max_depth=2),n_estimators = 50 , random_state=0 ) 
evaluation(final_model_AdaBoost, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall" , plot =False)

test set : 
[[2982 2349]
 [5397 9132]]
              precision    recall  f1-score   support

           1       0.36      0.56      0.44      5331
           2       0.80      0.63      0.70     14529

    accuracy                           0.61     19860
   macro avg       0.58      0.59      0.57     19860
weighted avg       0.68      0.61      0.63     19860

train set : 
[[12034  9363]
 [20757 37282]]
              precision    recall  f1-score   support

           1       0.37      0.56      0.44     21397
           2       0.80      0.64      0.71     58039

    accuracy                           0.62     79436
   macro avg       0.58      0.60      0.58     79436
weighted avg       0.68      0.62      0.64     79436

GRAPHIQUE NON DEMANDE


In [12]:
# TRAIN SET TEST SET
vingtiles(model = final_model_AdaBoost, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Adaboost.xlsx")

                 IDENT     PROBA
41557  415580010366670  0.318699
51872  518730010377392  0.318767
14631  146320010261695  0.319904
79066  790670010415017  0.320262
32764  327650010355036  0.320385
...                ...       ...
51980  519810010377558  0.670247
80462  804630010416496  0.670698
82246  822470010486983  0.672538
75982  759830010410948  0.678212
87203  872040010521403  0.754833

[99296 rows x 2 columns]
0.500216    193
0.499127    180
0.500984    156
0.499895    133
0.500940    104
           ... 
0.508560      1
0.501133      1
0.491389      1
0.500035      1
0.499623      1
Name: PROBA, Length: 63545, dtype: int64
                  PROBA
41557  (0.3186, 0.4959]
51872  (0.3186, 0.4959]
14631  (0.3186, 0.4959]
79066  (0.3186, 0.4959]
32764  (0.3186, 0.4959]
...                 ...
51980  (0.5047, 0.7548]
80462  (0.5047, 0.7548]
82246  (0.5047, 0.7548]
75982  (0.5047, 0.7548]
87203  (0.5047, 0.7548]

[99296 rows x 1 columns]
Top_mon              1     2  TOTAL
TRANCHES   

In [13]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_AdaBoost, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Adaboost.xlsx")

                     IDENT     PROBA
444570    3452750160001861  0.316018
202982    1036870050043677  0.318215
174910     756150050015605  0.318370
1134735  10354400370051903  0.318699
260655    1613600080004646  0.318857
...                    ...       ...
1119133  10198380370036301  0.678212
633736    5344410250006520  0.678492
641712    5424170250014471  0.678492
635652    5363570250008436  0.679091
132404     331090020023107  0.754833

[1092884 rows x 2 columns]
0.500216    2509
0.500984    2383
0.502029    1551
0.497325    1533
0.501632    1175
            ... 
0.492191       1
0.496387       1
0.502594       1
0.510512       1
0.506298       1
Name: PROBA, Length: 343191, dtype: int64
                    PROBA
444570   (0.3159, 0.4958]
202982   (0.3159, 0.4958]
174910   (0.3159, 0.4958]
1134735  (0.3159, 0.4958]
260655   (0.3159, 0.4958]
...                   ...
1119133  (0.5061, 0.7548]
633736   (0.5061, 0.7548]
641712   (0.5061, 0.7548]
635652   (0.5061, 0.7548]
132404   (0.5

In [ ]:
#############################################################
#                   MODELE 2 : RANDOM FOREST
#############################################################

In [ ]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'], 'max_depth' : [2,3,4,5],"max_features":[8,12,15,20,25,30] ,'n_estimators' : [20,50,100,200] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "recall", n_jobs=2)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_


{'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 25,
 'n_estimators': 100}

In [14]:
final_model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", criterion="entropy", max_depth=2, max_features=25, n_estimators = 100, random_state=0) 
evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[3337 1994]
 [6697 7832]]
              precision    recall  f1-score   support

           1       0.33      0.63      0.43      5331
           2       0.80      0.54      0.64     14529

    accuracy                           0.56     19860
   macro avg       0.56      0.58      0.54     19860
weighted avg       0.67      0.56      0.59     19860

train set : 
[[13040  8357]
 [26732 31307]]
              precision    recall  f1-score   support

           1       0.33      0.61      0.43     21397
           2       0.79      0.54      0.64     58039

    accuracy                           0.56     79436
   macro avg       0.56      0.57      0.53     79436
weighted avg       0.67      0.56      0.58     79436

GRAPHIQUE NON DEMANDE


In [15]:
# TRAIN SET TEST SET
vingtiles(model = final_model_RandomForestClassifier, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Random_Forest.xlsx")

                 IDENT     PROBA
21093  210940010337388  0.431003
81351  813520010417435  0.433345
45203  452040010370375  0.433345
85735  857360010519730  0.433664
93208  932090010530978  0.433664
...                ...       ...
59942  599430010392241  0.652199
85895  858960010519940  0.652199
15264  152650010262550  0.652199
3728    37290010049725  0.652199
30193  301940010350422  0.652199

[99296 rows x 2 columns]
0.522340    2486
0.462567    2360
0.511184    1756
0.463206    1390
0.463146     922
            ... 
0.508820       1
0.483883       1
0.445886       1
0.445106       1
0.447721       1
Name: PROBA, Length: 19585, dtype: int64
               PROBA
21093  (0.43, 0.447]
81351  (0.43, 0.447]
45203  (0.43, 0.447]
85735  (0.43, 0.447]
93208  (0.43, 0.447]
...              ...
59942  (0.56, 0.652]
85895  (0.56, 0.652]
15264  (0.56, 0.652]
3728   (0.56, 0.652]
30193  (0.56, 0.652]

[99296 rows x 1 columns]
Top_mon            1     2  TOTAL
TRANCHES                         
(0.4

In [16]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_RandomForestClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Random_Forest.xlsx")

                     IDENT     PROBA
256380    1570850080000371  0.431003
961849    8625540360039017  0.431003
1186301  10870060370103469  0.431003
904358    8050630340010526  0.431003
522399    4231040190038663  0.431003
...                    ...       ...
296421    1971260110014400  0.657206
107409      81140010007952  0.658674
1115545  10162500370032713  0.659533
636562    5372670250009346  0.663301
895764    7964690340001932  0.666665

[1092884 rows x 2 columns]
0.522340    35171
0.462567    30189
0.463206    16015
0.520539    11016
0.463146    10093
            ...  
0.525004        1
0.508264        1
0.517442        1
0.510307        1
0.503065        1
Name: PROBA, Length: 64803, dtype: int64
                  PROBA
256380    (0.43, 0.446]
961849    (0.43, 0.446]
1186301   (0.43, 0.446]
904358    (0.43, 0.446]
522399    (0.43, 0.446]
...                 ...
296421   (0.565, 0.667]
107409   (0.565, 0.667]
1115545  (0.565, 0.667]
636562   (0.565, 0.667]
895764   (0.565, 0.667]



In [ ]:
#############################################################
#                   MODELE 3 : BAGGING
#############################################################

In [ ]:
model_Bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,20, 2)), 'n_estimators' : [20, 50],
         'bootstrap':[True, False], "bootstrap_features": [True, False]}

grid_Bagging = GridSearchCV(model_Bagging, param, cv = 4, scoring = "recall", n_jobs=2)
resultat_grid_Bagging = grid_Bagging.fit(train_X_s, train_Y_s)
resultat_grid_Bagging.best_params_


{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'bootstrap': False,
 'bootstrap_features': False,
 'n_estimators': 20}

In [ ]:
final_model_BaggingClassifier = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced", criterion="gini", max_depth=2, random_state=0),bootstrap=False, bootstrap_features=False, n_estimators = 20) 
evaluation(final_model_BaggingClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[3351 1980]
 [6937 7592]]
              precision    recall  f1-score   support

           1       0.33      0.63      0.43      5331
           2       0.79      0.52      0.63     14529

    accuracy                           0.55     19860
   macro avg       0.56      0.58      0.53     19860
weighted avg       0.67      0.55      0.58     19860

train set : 
[[13136  8261]
 [27415 30624]]
              precision    recall  f1-score   support

           1       0.32      0.61      0.42     21397
           2       0.79      0.53      0.63     58039

    accuracy                           0.55     79436
   macro avg       0.56      0.57      0.53     79436
weighted avg       0.66      0.55      0.58     79436

GRAPHIQUE NON DEMANDE


In [ ]:
vingtiles(model = final_model_BaggingClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Decision_Tree_Classifier.xlsx")

                 IDENT     PROBA
0          10010000001  0.422535
54721  547220010381167  0.422535
54723  547240010381169  0.422535
54726  547270010381172  0.422535
54730  547310010381176  0.422535
...                ...       ...
26294  262950010344113  0.747443
94780  947810010532992  0.747443
5695    56960010244031  0.747443
2017    20180010026147  0.747443
33013  330140010355655  0.747443

[99296 rows x 2 columns]
0.422535    48457
0.529059    41073
0.644284     5282
0.747443     4484
Name: PROBA, dtype: int64


ValueError: ignored

In [ ]:
#############################################################
#                   MODELE 4 : EXTRA-TREES
#############################################################

In [ ]:
model_ExtraTreesClassifier = ExtraTreesClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'], 'max_depth' : [2,3,4,5],"max_features":[8,12,15,20,25,30] ,'n_estimators' : [20,50,100,200] }

grid_ExtraTreesClassifier = GridSearchCV(model_ExtraTreesClassifier, param, cv = 4, scoring = "recall", n_jobs=2)
resultat_grid_ExtraTreesClassifier = grid_ExtraTreesClassifier.fit(train_X_s, train_Y_s)
resultat_grid_ExtraTreesClassifier.best_params_



{'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 25,
 'n_estimators': 100}

In [17]:
final_model_ExtraTreesClassifier = ExtraTreesClassifier( criterion="entropy", max_depth=2, max_features=25, n_estimators=100 , class_weight="balanced", random_state=0) 
evaluation(final_model_ExtraTreesClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[3337 1994]
 [6784 7745]]
              precision    recall  f1-score   support

           1       0.33      0.63      0.43      5331
           2       0.80      0.53      0.64     14529

    accuracy                           0.56     19860
   macro avg       0.56      0.58      0.54     19860
weighted avg       0.67      0.56      0.58     19860

train set : 
[[13090  8307]
 [26999 31040]]
              precision    recall  f1-score   support

           1       0.33      0.61      0.43     21397
           2       0.79      0.53      0.64     58039

    accuracy                           0.56     79436
   macro avg       0.56      0.57      0.53     79436
weighted avg       0.66      0.56      0.58     79436

GRAPHIQUE NON DEMANDE


In [18]:
vingtiles(model = final_model_ExtraTreesClassifier, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Extra_Trees_Classifier.xlsx")

                 IDENT     PROBA
10385  103860010257272  0.435484
38910  389110010363695  0.435787
62181  621820010395560  0.435998
70769  707700010404698  0.436404
16027  160280010263592  0.436629
...                ...       ...
2075    20760010027058  0.648462
50923  509240010376412  0.648462
12520  125210010259455  0.648462
2071    20720010027021  0.648462
39949  399500010364851  0.648767

[99296 rows x 2 columns]
0.526498    1291
0.458107    1120
0.518145     931
0.523358     757
0.451825     686
            ... 
0.475616       1
0.471833       1
0.519736       1
0.534679       1
0.547145       1
Name: PROBA, Length: 21072, dtype: int64
                PROBA
10385  (0.434, 0.448]
38910  (0.434, 0.448]
62181  (0.434, 0.448]
70769  (0.434, 0.448]
16027  (0.434, 0.448]
...               ...
2075   (0.559, 0.649]
50923  (0.559, 0.649]
12520  (0.559, 0.649]
2071   (0.559, 0.649]
39949  (0.559, 0.649]

[99296 rows x 1 columns]
Top_mon            1     2  TOTAL
TRANCHES                  

In [19]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_ExtraTreesClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Extra_Trees_Classifier.xlsx")

                    IDENT     PROBA
605731   5064360220047969  0.434779
262502   1632070080006493  0.435114
1042770  9434750360119938  0.435259
1004467  9051720360081635  0.435468
326493   2271980120024467  0.435468
...                   ...       ...
356649   2573540130004633  0.648767
99860       5650010000403  0.648767
356791   2574960130004775  0.648767
1081999  9827040360159167  0.648767
895764   7964690340001932  0.650241

[1092884 rows x 2 columns]
0.526498    17792
0.458107    14518
0.523358    10259
0.451825     9102
0.451756     8892
            ...  
0.568415        1
0.579878        1
0.468235        1
0.511280        1
0.535116        1
Name: PROBA, Length: 72465, dtype: int64
                  PROBA
605731   (0.434, 0.447]
262502   (0.434, 0.447]
1042770  (0.434, 0.447]
1004467  (0.434, 0.447]
326493   (0.434, 0.447]
...                 ...
356649    (0.564, 0.65]
99860     (0.564, 0.65]
356791    (0.564, 0.65]
1081999   (0.564, 0.65]
895764    (0.564, 0.65]

[1092884 row

In [ ]:
#############################################################
#                   MODELE 5 : XGBoost (IL FAUT L'AMELIORER CAR GRIDSEARCHCV PREND TROP DE TEMPS)
#############################################################

In [20]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [39]:

model_XGBClassifier = XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8,
                                                  colsample_bytree=0.8, objective= 'binary:logistic', nthread=-1, seed=0)
param = { 'scale_pos_weight': [0.32, 0.33, 0.34, 0.35] } #list(np.linspace(0.10, 0.30, 2))   #'max_depth':range(3,10,2), 'min_child_weight':range(1,6,2),

grid_XGBClassifier = GridSearchCV(estimator = model_XGBClassifier, param_grid = param, scoring='recall', cv=5)
resultat_grid_XGBClassifier = grid_XGBClassifier.fit(train_X_s, train_Y_s)
resultat_grid_XGBClassifier.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 1,
 'n_estimators': 50,
 'objective': 'binary:logistic',
 'scale_pos_weight': 0.35,
 'seed': 0,
 'subsample': 0.8}

In [55]:
final_model_XGBClassifier = XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=3, min_child_weight=1, gamma=0, subsample=0.8,
                                                  colsample_bytree=0.8, objective= 'binary:logistic', nthread=2, scale_pos_weight=0.34, seed=0)
evaluation(final_model_XGBClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[3313 2018]
 [6270 8259]]
              precision    recall  f1-score   support

           1       0.35      0.62      0.44      5331
           2       0.80      0.57      0.67     14529

    accuracy                           0.58     19860
   macro avg       0.57      0.59      0.56     19860
weighted avg       0.68      0.58      0.61     19860

train set : 
[[13348  8049]
 [24432 33607]]
              precision    recall  f1-score   support

           1       0.35      0.62      0.45     21397
           2       0.81      0.58      0.67     58039

    accuracy                           0.59     79436
   macro avg       0.58      0.60      0.56     79436
weighted avg       0.68      0.59      0.61     79436

GRAPHIQUE NON DEMANDE


In [56]:
vingtiles_xgb(model = final_model_XGBClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_XGBoost.xlsx")

                 IDENT     PROBA
66003  660040010399617  0.227716
67823  678240010401567  0.227716
78844  788450010414774  0.227716
29727  297280010349859  0.232173
68075  680760010401859  0.236831
...                ...       ...
48128  481290010373476  0.885968
83091  830920010502100  0.885968
5615    56160010208685  0.885968
83536  835370010503920  0.886497
60221  602220010392936  0.888440

[99296 rows x 2 columns]
0.546209    385
0.495340    256
0.403445    226
0.550064    213
0.431736    197
           ... 
0.700457      1
0.412724      1
0.475223      1
0.412721      1
0.437500      1
Name: PROBA, Length: 58312, dtype: int64
                PROBA
66003  (0.227, 0.361]
67823  (0.227, 0.361]
78844  (0.227, 0.361]
29727  (0.227, 0.361]
68075  (0.227, 0.361]
...               ...
48128  (0.717, 0.888]
83091  (0.717, 0.888]
5615   (0.717, 0.888]
83536  (0.717, 0.888]
60221  (0.717, 0.888]

[99296 rows x 1 columns]
Top_mon            1     2  TOTAL
TRANCHES                         
(0.

In [ ]:
###################################################################################################################@
###################################################################################################################@
#                                   DECISION TREE CLASSIFIER  ----  AVEC VARIABLES QUALITATIVES
###################################################################################################################@
###################################################################################################################@

In [ ]:
####################################################################################
####################################################################################
#                   TEST SANS LES VARIABLES ILLUSTRATIVES DE SPAD
####################################################################################
####################################################################################

In [ ]:
# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F",
                "Onoma-Phase_Scl", "Onoma-Phase_CL"],axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )
data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe IFI+
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:

model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced" )
param = { 'max_depth' :list(range(2,20)), 'criterion': ["gini", "entropy"], 'max_features' : list(range(3,10,1)),
         'min_samples_split' : [0.05, 0.1, 0.4, 0.6, 0.8, 1] }

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "recall", n_jobs=2)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s, train_Y_s)
resultat_grid_DecisionTreeClassifier.best_params_


{'criterion': 'gini',
 'max_depth': 2,
 'max_features': 5,
 'min_samples_split': 0.05}

In [ ]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, criterion="gini", class_weight = "balanced", max_depth = 2, max_features=13, min_samples_split=0.05 )
evaluation(final_model_DecisionTreeClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 5180   151]
 [13889   640]]
              precision    recall  f1-score   support

           1       0.27      0.97      0.42      5331
           2       0.81      0.04      0.08     14529

    accuracy                           0.29     19860
   macro avg       0.54      0.51      0.25     19860
weighted avg       0.66      0.29      0.18     19860

train set : 
[[20819   578]
 [55499  2540]]
              precision    recall  f1-score   support

           1       0.27      0.97      0.43     21397
           2       0.81      0.04      0.08     58039

    accuracy                           0.29     79436
   macro avg       0.54      0.51      0.25     79436
weighted avg       0.67      0.29      0.18     79436

GRAPHIQUE NON DEMANDE


In [ ]:
# TRAIN SET TEST SET
vingtiles(model = final_model_DecisionTreeClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Decision_Tree_Classifier.xlsx")

                 IDENT     PROBA
56970  569710010384479  0.381666
9253    92540010256064  0.381666
52518  525190010378240  0.381666
60182  601830010392815  0.381666
22850  228510010339375  0.381666
...                ...       ...
29519  295200010349528  0.553444
94344  943450010532481  0.553444
69420  694210010403242  0.553444
44966  449670010370130  0.553444
49402  494030010374795  0.553444

[99296 rows x 2 columns]
0.501674    90709
0.553444     4678
0.381666     3909
Name: PROBA, dtype: int64


ValueError: ignored

In [ ]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_DecisionTreeClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Decision_Tree_Classifier.xlsx")